# 如何解析 JSON 输出
:::info 前提条件
本指南假定您已熟悉以下概念：- [聊天模型](/docs/concepts/chat_models)- [输出解析器](/docs/concepts/output_parsers)- [提示模板](/docs/concepts/prompt_templates)- [结构化输出](/docs/how_to/structured_output)- [将可运行对象链式组合](/docs/how_to/sequence/)
:::
虽然部分模型提供商支持[内置的结构化输出返回方式](/docs/how_to/structured_output)，但并非所有提供商都具备该功能。我们可以通过输出解析器实现以下流程：让用户通过提示词指定任意JSON格式模式，查询模型以获取符合该模式的输出，最终将该模式解析为JSON。
:::注意请记住，大型语言模型是存在漏洞的抽象层！您必须使用具备足够能力的LLM才能生成格式良好的JSON。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个标准的markdown格式翻译示例。

## 主要功能

1. 保持原文的markdown结构
2. 提供准确流畅的翻译
3. 仅输出翻译内容本身

### 注意事项

* 不添加额外说明文字
* 保持原有标题层级
* 保留列表和格式样式

> 引用内容也会被准确翻译

[链接文本](https://example.com) 同样会处理

```python
# 代码块不会翻译
print("Hello World")
```

[`JsonOutputParser`](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html) 是一个内置选项，用于提示并解析JSON输出。虽然它在功能上与 [`PydanticOutputParser`](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.pydantic.PydanticOutputParser.html) 类似，但它还支持流式返回部分JSON对象。
以下是与 [Pydantic](https://docs.pydantic.dev/) 结合使用的示例，可便捷地声明预期模式：

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

请注意，我们正在将解析器中的 `format_instructions` 直接传递到提示词中。您可以（也应该）尝试在提示词的其他部分添加自己的格式提示，以增强或替换默认指令：

In [3]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

## 流式传输
如上所述，`JsonOutputParser`与`PydanticOutputParser`的一个关键区别在于：`JsonOutputParser`输出解析器支持流式传输部分数据块。具体效果如下：

In [4]:
for s in chain.stream({"query": joke_query}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why couldn'}
{'setup': "Why couldn't"}
{'setup': "Why couldn't the"}
{'setup': "Why couldn't the bicycle"}
{'setup': "Why couldn't the bicycle stand"}
{'setup': "Why couldn't the bicycle stand up"}
{'setup': "Why couldn't the bicycle stand up by"}
{'setup': "Why couldn't the bicycle stand up by itself"}
{'setup': "Why couldn't the bicycle stand up by itself?"}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': ''}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'Because it was two tired'}
{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline'

## 不使用 Pydantic
你也可以在不使用 Pydantic 的情况下使用 `JsonOutputParser`。这会提示模型返回 JSON 格式的数据，但不会提供关于具体模式（schema）的详细信息。

In [5]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'response': "Sure! Here's a joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired!"}

## 后续步骤
你现在已经学会了一种让模型返回结构化 JSON 的提示方法。接下来，请查看[获取结构化输出的综合指南](/docs/how_to/structured_output)了解其他技术。